In [15]:
import pandas as pd
import pickle
from scipy.stats import poisson

In [16]:
dict_table = pickle.load(open('dict_table','rb'))
df_historical_data = pd.read_csv('clean_fifa_worldcup_matches.csv')
df_fixture = pd.read_csv('clean_fifa_worldcup_fixture.csv')

## 1.- Calcular Team Strength

In [17]:
#Dividir df entre df_home y df_away

df_home = df_historical_data[['HomeTeam','HomeGoals', 'AwayGoals']]
df_away = df_historical_data[['AwayTeam','HomeGoals', 'AwayGoals']]


In [18]:
#Cambiar normbres a las columnas:

df_home = df_home.rename(columns={'HomeTeam': 'Team', 'HomeGoals':'GoalsScored', 'AwayGoals':'GoalsConceded'})
df_away = df_away.rename(columns={'AwayTeam': 'Team', 'HomeGoals':'GoalsConceded', 'AwayGoals':'GoalsScored'})

In [23]:
#Concatenar df_home y df_away, hacer gruop por team y calcula promedio -> Team strength

df_team_strength = pd.concat([df_home, df_away], ignore_index=True).groupby('Team').mean()
df_team_strength

,GoalsScored,GoalsConceded
Team,,
Algeria,1.000000,1.461538
Angola,0.333333,0.666667
Argentina,1.691358,1.148148
Australia,0.812500,1.937500
Austria,1.482759,1.620690
...,...,...
Uruguay,1.553571,1.321429
Wales,0.800000,0.800000
West Germany,2.112903,1.241935


## 2.-Función de predicción utilizando distribución de Poisson

In [32]:
def predict_points(home,away):
    if home in df_team_strength.index and away in df_team_strength.index:
        #goals scored * goals conceded
        lamb_home = df_team_strength.at[home,'GoalsScored'] * df_team_strength.at[away, 'GoalsConceded']
        lamb_away = df_team_strength.at[away,'GoalsScored'] * df_team_strength.at[home, 'GoalsConceded']
        
        prob_home, prob_away, prob_draw = 0,0,0
        
        for x in range(0,11):  #number of goals home team
            for y in range(0,11): #number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                
                if x ==y:
                    prob_draw += p
                elif x>y:
                    prob_home +=p
                else:
                    prob_away +=p
                    
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
    else:
        return(0,0)

## 2.1.-Testear función


In [42]:
#Testeamos partidos: Argentina-Mexico, Inglaterra-US, Qatar-Ecuador:

predict_points('Argentina', 'Mexico')
predict_points('England', 'United States')
predict_points('Qatar (H)', 'Ecuador')

(0, 0)

## 3.-Prediciendo Mundial

### 3.1.- Fase de grupos:

In [44]:
#dividiendo fixture en grupo, octavos, cuartos, etc.

df_fixture_group_48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semi = df_fixture[60:62].copy()
df_fixture_final =df_fixture[62:].copy()

In [55]:
#Correr todos los partidos de la fase de grupos y actualizar las tablas de cada grupo

for group in dict_table:
    teams_in_group = dict_table[group]['Team'].values
    df_fixture_group_6 = df_fixture_group_48[df_fixture_group_48['home'].isin(teams_in_group)]
    
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away  = predict_points(home,away)
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away
        
    dict_table[group] = dict_table[group].sort_values('Pts', ascending = False).reset_index()
    dict_table[group] = dict_table[group][['Team', 'Pts']]
    dict_table[group] = dict_table[group].round(0)

In [61]:
#Mostrar tabla actualizada:
dict_table['Grupo A']
dict_table['Grupo B']
dict_table['Grupo C']
#etc

,Team,Pts
0,Argentina,7.0
1,Poland,6.0
2,Mexico,4.0
3,Saudi Arabia,1.0


### 3.2.- Octavos

In [62]:
#octavos: df_fixture_knockout
df_fixture_knockout

,home,score,away,year
48,Winners Group A,Match 49,Runners-up Group B,2022
49,Winners Group C,Match 50,Runners-up Group D,2022
50,Winners Group D,Match 52,Runners-up Group C,2022
51,Winners Group B,Match 51,Runners-up Group A,2022
52,Winners Group E,Match 53,Runners-up Group F,2022
53,Winners Group G,Match 54,Runners-up Group H,2022
54,Winners Group F,Match 55,Runners-up Group E,2022
55,Winners Group H,Match 56,Runners-up Group G,2022


In [77]:
#Cambiando nombres diccionario para que coincidan

dict_table['Group A'] = dict_table['Grupo A']
dict_table.pop('Grupo A')
dict_table['Group B'] = dict_table['Grupo B']
dict_table.pop('Grupo B')
dict_table['Group C'] = dict_table['Grupo C']
dict_table.pop('Grupo C')
dict_table['Group D'] = dict_table['Grupo D']
dict_table.pop('Grupo D')
dict_table['Group E'] = dict_table['Grupo E']
dict_table.pop('Grupo E')
dict_table['Group F'] = dict_table['Grupo F']
dict_table.pop('Grupo F')
dict_table['Group G'] = dict_table['Grupo G']
dict_table.pop('Grupo G')
dict_table['Group H'] = dict_table['Grupo H']
dict_table.pop('Grupo H')
dict_table

KeyError: 'Grupo A'

In [78]:
#Actualizar el fixture de octavos con el 1 puesto (group winner) y segundo puesto (runners up)

for group in dict_table:
    group_winner = dict_table[group].loc[0, 'Team']
    runners_up = dict_table[group].loc[1, 'Team']
    
    df_fixture_knockout.replace({f'Winners {group}': group_winner, f'Runners-up {group}': runners_up}, inplace = True)
    
df_fixture_knockout['winner'] = '?'

In [79]:
df_fixture_knockout

,home,score,away,year,winner
48,Netherlands,Match 49,Wales,2022,?
49,Argentina,Match 50,Denmark,2022,?
50,France,Match 52,Poland,2022,?
51,England,Match 51,Senegal,2022,?
52,Germany,Match 53,Belgium,2022,?
53,Brazil,Match 54,Uruguay,2022,?
54,Croatia,Match 55,Spain,2022,?
55,Portugal,Match 56,Switzerland,2022,?


In [80]:
#Crear función get_winner:

def get_winner(df_fixture_updated):
    for index,row in df_fixture_updated.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        if points_home > points_away:
            winner = home
        else:
            winner = away
        
        df_fixture_updated.loc[index, 'winner'] = winner
        
    return df_fixture_updated

In [81]:
get_winner(df_fixture_knockout)

,home,score,away,year,winner
48,Netherlands,Match 49,Wales,2022,Netherlands
49,Argentina,Match 50,Denmark,2022,Argentina
50,France,Match 52,Poland,2022,France
51,England,Match 51,Senegal,2022,England
52,Germany,Match 53,Belgium,2022,Germany
53,Brazil,Match 54,Uruguay,2022,Brazil
54,Croatia,Match 55,Spain,2022,Spain
55,Portugal,Match 56,Switzerland,2022,Portugal


### 3.3.- Cuartos de final

In [85]:
#Crear función update_table

def update_table(df_fixture_round_1, df_fixture_round_2):
    for index, row in df_fixture_round_1.iterrows():
        winner = df_fixture_round_1.loc[index, 'winner']
        match = df_fixture_round_1.loc[index, 'score']
        df_fixture_round_2.replace({f'Winners {match}':winner}, inplace = True)
    df_fixture_round_2['winner'] = '?'
    return df_fixture_round_2



In [87]:
#Creando la tabla de cuartos:

update_table(df_fixture_knockout, df_fixture_quarter)

,home,score,away,year,winner
56,Germany,Match 58,Brazil,2022,?
57,Netherlands,Match 57,Argentina,2022,?
58,Spain,Match 60,Portugal,2022,?
59,England,Match 59,France,2022,?


In [88]:
#Ver ganador:
get_winner(df_fixture_quarter)

,home,score,away,year,winner
56,Germany,Match 58,Brazil,2022,Brazil
57,Netherlands,Match 57,Argentina,2022,Netherlands
58,Spain,Match 60,Portugal,2022,Portugal
59,England,Match 59,France,2022,France


### 3.4.- Semifinal

In [89]:
#Creando la tabla de semis:

update_table(df_fixture_quarter, df_fixture_semi)

,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,?
61,France,Match 62,Portugal,2022,?


In [90]:
#Comprobando ganadores:

get_winner(df_fixture_semi)

,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,Brazil
61,France,Match 62,Portugal,2022,France


### 3.5.- Final

In [91]:
update_table(df_fixture_semi, df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,?
63,Brazil,Match 64,France,2022,?


In [92]:
get_winner(df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,Losers Match 62
63,Brazil,Match 64,France,2022,Brazil
